# Maritime Choke Point Trends Monitor

The objective of this analysis is to examine the impact of the Red Sea Conflict on maritime trade statistics derived from AIS data.  

We process transit calls and estimated trade volume from the IMF's [PortWatch](https://portwatch.imf.org/) platform for key ports of interest, and then produce some charts to inspect trends and calculate percentage changes.

In [2]:
import os
import sys
from os.path import join


import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

import git

git_repo = git.Repo(os.getcwd(), search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")
sys.path.append(join(git_root, "src", "red-sea-monitoring"))
from utils import *

# For plotting
from plotnine import *
from mizani.breaks import date_breaks
from mizani.formatters import date_format, comma_format
import plotnine

plotnine.options.figure_size = (10, 8)

output_dir = r"C:\Users\WB514197\OneDrive - WBG\GOST_Deliverables\Red Sea\Data"
charts_dir = join(git_root, "reports")

In [3]:
%load_ext autoreload
%autoreload 2

## Ports

In [4]:
ports = get_ports()

In [5]:
countries = ["Egypt", "Yemen", "Djibouti", "Jordan", "Saudi Arabia"]

In [6]:
ports_sel = ports.loc[ports.country.isin(countries)].copy()

In [7]:
ports_sel.sort_values("country", inplace=True)

In [8]:
del ports

In [8]:
ports_sel.loc[:, "geometry"] = ports_sel.apply(lambda x: Point(x.lon, x.lat), axis=1)

In [9]:
ports_gdf = gpd.GeoDataFrame(ports_sel, geometry="geometry", crs="EPSG:4326")

In [10]:
# ports_gdf.explore()

In [11]:
# ports_gdf.to_file('ports_sel.geojson', driver='GeoJSON')
del ports_gdf

In [9]:
ports_red_sea = gpd.read_file(
    join(git_root, "data", "red_sea_ports.geojson"), driver="GeoJSON"
)
# ports_red_sea = pd.read_csv('red_sea_ports.csv')

In [10]:
# ports_red_sea.explore()

In [11]:
ports_red_sea[["country", "portname", "portid"]]

,country,portname,portid
0,Djibouti,Djibouti,port294
1,Egypt,El-Adabiya,port321
2,Egypt,Safaga,port191
3,Egypt,North Ain Sukhna Port,port828
4,Egypt,As Suways,port71
5,Jordan,Al Aqabah,port19
6,Saudi Arabia,Duba Bulk Plant Tanker Terminal,port305
7,Saudi Arabia,Rabigh,port1081
8,Saudi Arabia,King Fahd Port,port570
9,Saudi Arabia,Duba,port304


### Trade Data

In [15]:
ports = list(ports_red_sea.portid)

In [17]:
df_ports = get_port_data(ports)

In [ ]:
df_ports.to_csv(join(output_dir, "ports_data.csv"), index=False)

## Data Analysis

In [51]:
df = pd.read_csv(join(output_dir, "ports_data.csv"))
df.date = pd.to_datetime(df.date)

In [52]:
df = df.loc[df.date >= "2019-01-01"].copy()

In [53]:
df = df[
    [
        "portname",
        "portcalls_cargo",
        "portcalls_tanker",
        "portcalls",
        "import_cargo",
        "export_cargo",
        "import_tanker",
        "export_tanker",
        "import",
        "export",
        "date",
    ]
].copy()

In [54]:
# df.loc[:, "ym"] = df.date.dt.strftime('%Y-%m-%d')
# df.loc[:, "w"] = df.date.dt.strftime('%W')

In [55]:
# df = df.groupby('portname')[['portcalls_cargo', 'portcalls_tanker', 'portcalls', 'import_cargo', 'export_cargo', 'import_tanker', 'export_tanker', 'import', 'export', 'date']].rolling(7, center=True, min_periods=1, on='date').mean()
df = (
    df.groupby("portname")[
        [
            "portcalls_cargo",
            "portcalls_tanker",
            "portcalls",
            "import_cargo",
            "export_cargo",
            "import_tanker",
            "export_tanker",
            "import",
            "export",
            "date",
        ]
    ]
    .resample("MS", on="date")
    .sum()
    .reset_index()
)
# df.reset_index(inplace=True)
# df.drop('level_1', axis=1, inplace=True)
df.loc[:, "ymd"] = df.date.dt.strftime("%Y-%m-%d")
# df.loc[:, "w"] = df.date.dt.strftime('%W')
df.loc[:, "m"] = df.date.dt.strftime("%m")
# df.loc[:, "w"] = df.date.dt.strftime('%W')

In [56]:
df.head()

,portname,date,portcalls_cargo,portcalls_tanker,portcalls,import_cargo,export_cargo,import_tanker,export_tanker,import,export,ymd,m
0,Aden,2019-01-01,35,7,42,211348.052875,1848.225266,61496.118007,3160.095388,272844.170882,5008.320654,2019-01-01,01
1,Aden,2019-02-01,43,5,48,306744.763449,10160.227084,42407.618684,6320.190776,349152.382132,16480.417860,2019-02-01,02
2,Aden,2019-03-01,44,2,46,307431.355733,5396.423353,0.000000,2976.724747,307431.355733,8373.148100,2019-03-01,03
3,Aden,2019-04-01,42,9,51,418376.771857,6930.024697,101798.953706,3160.095388,520175.725563,10090.120085,2019-04-01,04
4,Aden,2019-05-01,33,8,41,335058.691905,279.225541,99985.495619,0.000000,435044.187525,279.225541,2019-05-01,05


### Plot Transit Calls Historical

In [57]:
# start_reference_date = "2022-01-01"
conflict_date = "2023-10-07"
crisis_date = "2023-11-17"

In [58]:
df_filt = df.loc[(df.date >= "2023-01-01")].copy()

In [59]:
charts_by_port_dir = join(charts_dir, "ports")
if not os.path.exists(charts_by_port_dir):
    os.makedirs(charts_by_port_dir, mode=0o777)

In [60]:
for port in df_filt.portname.unique()[0:1]:
    port_info = ports_red_sea.loc[ports_red_sea.portname == port].iloc[0]
    country = port_info.country
    port_id = port_info.portid
    country = ports_red_sea.loc[ports_red_sea.portname == port, "country"].values[0]
    df_port = df_filt.loc[df_filt.portname == port].copy()
    df_port = df_port.melt(
        id_vars="date",
        value_vars=["import", "export"],
        var_name="direction",
        value_name="trade",
    )
    df_port.loc[:, "direction"] = df_port.direction.str.capitalize()
    p0 = (
        ggplot(df_port, aes(x="date", y="trade", fill="direction"))  #
        + geom_bar(alpha=3 / 4, stat="identity", position="dodge2")  #  fill="lightblue"
        # + geom_line(aes(x="date", y="export"), alpha=1, color="darkred")
        + geom_vline(xintercept=conflict_date, linetype="dashed", color="black")
        + geom_vline(xintercept=crisis_date, linetype="dashed", color="black")
        + labs(
            x="",
            y="Metric Tons",
            title=f"Weekly Trade Volume - {port}, {country}",
            fill="Trade Flow",
        )
        + theme_minimal()
        + theme(text=element_text(family="Roboto"))
        + scale_x_datetime(breaks=date_breaks("1 month"), labels=date_format("%Y-%m"))
        + scale_y_continuous(labels=comma_format())
        # + scale_fill_brewer(type='qual', palette=1)
        + theme(axis_text_x=element_text(rotation=45, hjust=1))
        + theme(legend_position="bottom")
    )
    p0
    # p0.save(filename=join(charts_by_port_dir, f'estimated-trade-{port_id}.jpeg'), dpi=300)

In [62]:
# p0

Periods
- **Baseline**: 2021, 2022, 2023 (January 1st – October 6th)
- **Middle East Conflict**: 2023 (October 7th - November 16th)
- **Red Sea Crisis**: November 17th - January 31st, 2024

### Calculate Reference Values

In [63]:
start_reference_date = "2022-01-01"
conflict_date = "2023-10-07"
crisis_date = "2023-11-17"

In [64]:
df_ref = df.loc[(df.date >= start_reference_date) & (df.date < conflict_date)].copy()

In [65]:
df_ref = df_ref.groupby(["portname", "m"])[
    [
        "portcalls_cargo",
        "portcalls_tanker",
        "portcalls",
        "import_cargo",
        "export_cargo",
        "import_tanker",
        "export_tanker",
        "import",
        "export",
    ]
].mean()

In [66]:
df_ref.reset_index(inplace=True)

In [67]:
df_ref.rename(
    columns={
        "portcalls_cargo": "portcalls_cargo_ref",
        "portcalls_tanker": "portcalls_tanker_ref",
        "portcalls": "portcalls_ref",
        "import_cargo": "import_cargo_ref",
        "export_cargo": "export_cargo_ref",
        "import_tanker": "import_tanker_ref",
        "export_tanker": "export_tanker_ref",
        "import": "import_ref",
        "export": "export_ref",
    },
    inplace=True,
)

In [68]:
df_filt = df.loc[(df.date >= "2023-01-01")].copy()

In [69]:
df_filt = df_filt.merge(df_ref, on=["portname", "m"], how="left", validate="m:1")

In [70]:
# df_filt.loc[:, "export_pct_ch"] = df_filt.apply(lambda x: (x.export-x.export_ref) / (x.export_ref), axis=1)
# df_filt.loc[:, "import_pct_ch"] = df_filt.apply(lambda x: (x['import']-x.import_ref) / (x.import_ref), axis=1)

In [177]:
charts_by_port_dir = join(charts_dir, "ports-ref-monthly")
if not os.path.exists(charts_by_port_dir):
    os.makedirs(charts_by_port_dir, mode=0o777)

In [179]:
for port in df_filt.portname.unique():
    port_info = ports_red_sea.loc[ports_red_sea.portname == port].iloc[0]
    country = port_info.country
    port_id = port_info.portid
    country = ports_red_sea.loc[ports_red_sea.portname == port, "country"].values[0]
    df_port = df_filt.loc[df_filt.portname == port].copy()
    df_port_copy = df_port.copy()
    df_port = df_port.melt(
        id_vars="date",
        value_vars=["import", "export"],
        var_name="direction",
        value_name="trade",
    )
    df_port.loc[:, "direction"] = df_port.direction.str.capitalize()
    p0 = (
        ggplot(df_port_copy, aes(x="date", y="import_ref"))  #
        + geom_smooth(
            mapping=aes(x="date", y="import_ref"), color="teal", size=0.4, alpha=3 / 4
        )
        + geom_smooth(
            mapping=aes(x="date", y="export_ref"), color="red", size=0.4, alpha=3 / 4
        )
        + geom_bar(
            mapping=aes(x="date", y="trade", fill="direction"),
            data=df_port,
            alpha=3 / 4,
            stat="identity",
            position="dodge2",
        )  #  fill="lightblue"
        # + geom_line(aes(x="date", y="export"), alpha=1, color="darkred")
        + geom_vline(xintercept=conflict_date, linetype="dashed", color="black")
        + geom_vline(xintercept=crisis_date, linetype="dashed", color="black")
        + labs(
            x="",
            y="Metric Tons",
            title=f"Monthly Trade Volume - {port}, {country}",
            fill="Trade Flow",
        )
        + theme_minimal()
        + theme(text=element_text(family="Roboto"))
        + scale_x_datetime(breaks=date_breaks("1 month"), labels=date_format("%Y-%m"))
        + scale_y_continuous(labels=comma_format())
        # + scale_fill_brewer(type='qual', palette=1)
        + theme(axis_text_x=element_text(rotation=45, hjust=1))
        + theme(legend_position="bottom")
    )
    p0.save(
        filename=join(charts_by_port_dir, f"estimated-trade-{port_id}.jpeg"), dpi=300
    )

c:\WBG\Anaconda3\envs\rtmis\Lib\site-packages\plotnine\ggplot.py:587: PlotnineWarning: Saving 10 x 8 in image.
c:\WBG\Anaconda3\envs\rtmis\Lib\site-packages\plotnine\ggplot.py:588: PlotnineWarning: Filename: C:/Users/WB514197/Repos/red-sea-monitoring\reports\ports-ref-monthly\estimated-trade-port9.jpeg
c:\WBG\Anaconda3\envs\rtmis\Lib\site-packages\plotnine\stats\smoothers.py:330: PlotnineWarning: Confidence intervals are not yet implemented for lowess smoothings.
c:\WBG\Anaconda3\envs\rtmis\Lib\site-packages\plotnine\stats\smoothers.py:330: PlotnineWarning: Confidence intervals are not yet implemented for lowess smoothings.
c:\WBG\Anaconda3\envs\rtmis\Lib\site-packages\plotnine\ggplot.py:587: PlotnineWarning: Saving 10 x 8 in image.
c:\WBG\Anaconda3\envs\rtmis\Lib\site-packages\plotnine\ggplot.py:588: PlotnineWarning: Filename: C:/Users/WB514197/Repos/red-sea-monitoring\reports\ports-ref-monthly\estimated-trade-port18.jpeg
c:\WBG\Anaconda3\envs\rtmis\Lib\site-packages\plotnine\stats\sm

### Summary

In [74]:
conflict_date, crisis_date

('2023-10-07', '2023-11-17')

In [77]:
df.loc[:, "period"] = ""

In [78]:
df.loc[
    (df.date >= start_reference_date) & (df.date < crisis_date), "period"
] = "Reference"
df.loc[
    (df.date >= conflict_date) & (df.date < crisis_date), "period"
] = "Middle East Conflict"
df.loc[(df.date >= crisis_date), "period"] = "Red Sea Crisis"

In [87]:
df = df.merge(ports_red_sea[["portname", "country"]], on="portname")

In [101]:
df_agg = (
    df.loc[df.period != ""]
    .groupby(["country", "portname", "period"])[
        [
            "import_cargo",
            "export_cargo",
            "import_tanker",
            "export_tanker",
            "import",
            "export",
        ]
    ]
    .mean()
)

In [102]:
# change order of rows
df_agg = df_agg.reindex(
    ["Reference", "Middle East Conflict", "Red Sea Crisis"], level=2
)

In [103]:
df_agg.head()

import_cargo  export_cargo  \
country  portname  period                                              
Djibouti Djibouti  Reference             496247.680293  36377.760091   
                   Middle East Conflict  506080.778169  49467.517026   
                   Red Sea Crisis        401003.352096  77485.393260   
Egypt    As Suways Reference               4647.839993   5132.097389   
                   Middle East Conflict    5706.168006      0.000000   

                                         import_tanker  export_tanker  \
country  portname  period                                               
Djibouti Djibouti  Reference             158453.719309   12919.346749   
                   Middle East Conflict  120204.648986    4082.797276   
                   Red Sea Crisis        139391.934181   17144.853873   
Egypt    As Suways Reference                782.534881    1126.581524   
                   Middle East Conflict       0.000000       0.000000   

                                                import        export  
country  portname  period                                             
Djibouti Djibouti  Reference             654701.399602  49297.106841  
                   Middle East Conflict  626285.427155  53550.314301  
                   Red Sea Crisis        540395.286277  94630.247133  
Egypt    As Suways Reference               5430.374874   6258.678912  
                   Middle East Conflict    5706.168006      0.000000

In [105]:
table = df_agg.copy()
# format column numbers to 2 decimal places only for first three columns
table = table.applymap(lambda x: "{:,.0f}".format(x))
# df_agg = df_agg.applymap(lambda x: "{:.2f}".format(x))
# table.iloc[:, :3] = table.iloc[:, :3].applymap(lambda x: "{:.2f}".format(x))

# format last column numbers to thousands
# table.loc[:, "capacity"] = table.capacity.apply(lambda x: "{:,.0f}".format(x))

table.rename(
    columns={
        "import_cargo": "Cargo Import",
        "export_cargo": "Cargo Export",
        "import_tanker": "Tanker Import",
        "export_tanker": "Tanker Export",
        "import": "Total Import",
        "export": "Total Export",
    },
    inplace=True,
)
# table
# table.transpose()

C:\Users\WB514197\AppData\Local\Temp\ipykernel_5064\1394900831.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.


In [111]:
# table.reset_index().rename(columns={'country': 'Country', 'portname': 'Port', 'period': 'Period'})

In [116]:
out = (
    table.reset_index()
    .rename(columns={"country": "Country", "portname": "Port", "period": "Period"})
    .to_markdown(index=False)
)

In [118]:
# print(out)

In [138]:
# aois = list(df_filt.portname.unique())
countries = list(df.country.unique())

In [131]:
df_agg_copy = df_agg.copy()

In [125]:
# df_agg_copy.reset_index(inplace=True)

In [135]:
# df_agg_copy.loc[("Yemen"),:]

In [150]:
# get list of first part of index from multi index
df_country.index.get_level_values(0).unique()

Index(['Aden', 'Al Ahmadi', 'Al Mukalla'], dtype='object', name='portname')

In [158]:
res = []
for country in countries:
    df_country = df_agg_copy.loc[(country), :].copy()
    aois = df_country.index.get_level_values(0).unique()
    for aoi in aois:
        df_sub = df_country.loc[(aoi), :].transpose().copy()
        df_sub.loc[:, "Middle East Conflict"] = (
            df_sub.loc[:, "Middle East Conflict"] - df_sub.loc[:, "Reference"]
        ) / df_sub.loc[:, "Reference"]
        df_sub.loc[:, "Red Sea Crisis"] = (
            df_sub.loc[:, "Red Sea Crisis"] - df_sub.loc[:, "Reference"]
        ) / df_sub.loc[:, "Reference"]
        df_sub2 = df_sub.transpose()
        df_sub2.drop("Reference", inplace=True)
        df_sub2.loc[:, "portname"] = aoi
        df_sub2.loc[:, "country"] = country
        res.append(df_sub2)

In [170]:
df_agg_pct = pd.concat(res)

In [171]:
df_agg_pct.reset_index(inplace=True)
df_agg_pct.set_index(["country", "portname", "period"], inplace=True)

# format columns as pct
df_agg_pct = df_agg_pct.applymap(lambda x: "{:.2%}".format(x))
df_agg_pct.rename(
    columns={
        "import_cargo": "Cargo Import",
        "export_cargo": "Cargo Export",
        "import_tanker": "Tanker Import",
        "export_tanker": "Tanker Export",
        "import": "Total Import",
        "export": "Total Export",
    },
    inplace=True,
)
# df_agg_pct

C:\Users\WB514197\AppData\Local\Temp\ipykernel_5064\3578456565.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.


In [172]:
out = (
    df_agg_pct.reset_index()
    .rename(columns={"country": "Country", "portname": "Port", "period": "Period"})
    .to_markdown(index=False)
)

In [173]:
print(out)

| Country      | Port                            | Period               | Cargo Import   | Cargo Export   | Tanker Import   | Tanker Export   | Total Import   | Total Export   |
|:-------------|:--------------------------------|:---------------------|:---------------|:---------------|:----------------|:----------------|:---------------|:---------------|
| Yemen        | Aden                            | Middle East Conflict | -24.87%        | -67.76%        | -28.57%         | -78.48%         | -25.44%        | -71.56%        |
| Yemen        | Aden                            | Red Sea Crisis       | -41.72%        | -69.11%        | -47.26%         | -100.00%        | -42.58%        | -80.06%        |
| Yemen        | Al Ahmadi                       | Middle East Conflict | 71.08%         | -23.33%        | 27.47%          | 506.90%         | 59.64%         | 60.55%         |
| Yemen        | Al Ahmadi                       | Red Sea Crisis       | 50.99%         | -73.16%        | -4